# CS 282
### Programming Assignment 1
#### Item 3

Jan Lendl R. Uy

2019-00312

#### Sytem Specifications
- macOS Sequoia 15.0.1
- Macbook Air M1 (ARM), 8-Core CPU and 8-Core GPU

#### Notes when Running the Notebook
- Image Viewer
  - Click on the image to see pixel coordinates and BGR values.
  - Press 'q' to terminate OpenCV2 windows.
- OpenCV2 windows tend to be buggy when attempted to be closed. They are best terminated through the following:
  - For macOS: Click 'Force Quit' in Activity Monitor
  - For Windows: Click 'End Task' in Task Manager

In [1]:
import cv2
import numpy as np
from PIL import Image

In [2]:
def read_image(image_path):
    """Read image in color format for BGR values"""
    try:
        # Try reading with OpenCV first
        img = cv2.imread(image_path)
        if img is not None:
            return img
        
        # If OpenCV fails, try with PIL
        pil_img = Image.open(image_path)
        # Convert to numpy array
        img = np.array(pil_img)
        # If image is RGB, convert to BGR for OpenCV
        if len(img.shape) == 3 and img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        return img
    except Exception as e:
        print(f"Error reading image: {e}")
        return None

class ImageClickDisplay:
    def __init__(self, image_path):
        self.image = read_image(image_path)
        if self.image is None:
            raise ValueError("Could not read image")
        self.display_image = self.image.copy()
        
        # Create window and set mouse callback
        cv2.namedWindow('Image')
        cv2.setMouseCallback('Image', self.mouse_callback)
        
        # Font settings
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.font_scale = 0.5
        self.font_thickness = 1
        
        # Colors
        self.bg_color = (40, 40, 40)
        self.text_color = (255, 255, 255)
        self.highlight_color = (0, 165, 255)  # Orange in BGR
        
        # For storing last click position
        self.last_click = None
        
    def draw_rounded_rectangle(self, img, top_left, bottom_right, color, radius=10, thickness=-1):
        """Draw a rounded rectangle on the image"""
        # Draw the main rectangle
        top_right = (bottom_right[0], top_left[1])
        bottom_left = (top_left[0], bottom_right[1])
        
        cv2.rectangle(img, 
                     (top_left[0] + radius, top_left[1]), 
                     (bottom_right[0] - radius, bottom_right[1]), 
                     color, thickness)
        cv2.rectangle(img, 
                     (top_left[0], top_left[1] + radius), 
                     (bottom_right[0], bottom_right[1] - radius), 
                     color, thickness)
        
        # Draw the four corner circles
        cv2.circle(img, (top_left[0] + radius, top_left[1] + radius), radius, color, thickness)
        cv2.circle(img, (top_right[0] - radius, top_right[1] + radius), radius, color, thickness)
        cv2.circle(img, (bottom_left[0] + radius, bottom_left[1] - radius), radius, color, thickness)
        cv2.circle(img, (bottom_right[0] - radius, bottom_right[1] - radius), radius, color, thickness)
        
    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            try:
                # Get BGR values at clicked point and ensure they're integers
                bgr = self.image[y, x]
                b = int(bgr[0])
                g = int(bgr[1])
                r = int(bgr[2])
                
                # Create copy of original image
                self.display_image = self.image.copy()
                
                # Store click position
                self.last_click = (x, y)
                
                # Draw crosshair at clicked point
                cv2.line(self.display_image, (x-10, y), (x+10, y), self.highlight_color, 2)
                cv2.line(self.display_image, (x, y-10), (x, y+10), self.highlight_color, 2)
                
                # Create info panel
                panel_width = 200
                panel_height = 90
                
                # Position the panel to stay within image bounds
                panel_x = min(x + 10, self.image.shape[1] - panel_width - 10)
                panel_y = min(y + 10, self.image.shape[0] - panel_height - 10)
                
                # Add semi-transparent overlay
                overlay = self.display_image.copy()
                
                # Draw rounded rectangle for background
                self.draw_rounded_rectangle(
                    overlay, 
                    (panel_x, panel_y), 
                    (panel_x + panel_width, panel_y + panel_height), 
                    self.bg_color
                )
                
                # Apply transparency
                alpha = 0.7
                cv2.addWeighted(overlay, alpha, self.display_image, 1 - alpha, 0, self.display_image)
                
                # Draw colored swatch of the selected pixel
                swatch_size = 30
                swatch_x = panel_x + panel_width - swatch_size - 10
                swatch_y = panel_y + 15
                
                # Create color tuple for OpenCV
                pixel_color = (b, g, r)
                
                # Draw the color swatch
                cv2.rectangle(self.display_image, 
                             (swatch_x, swatch_y), 
                             (swatch_x + swatch_size, swatch_y + swatch_size), 
                             pixel_color, 
                             -1)
                cv2.rectangle(self.display_image, 
                             (swatch_x, swatch_y), 
                             (swatch_x + swatch_size, swatch_y + swatch_size), 
                             (255, 255, 255), 
                             1)
                
                # Draw text
                cv2.putText(self.display_image, "PIXEL INFO", 
                           (panel_x + 10, panel_y + 20), 
                           self.font, self.font_scale, self.highlight_color, self.font_thickness)
                
                cv2.putText(self.display_image, f"X, Y: ({x}, {y})", 
                           (panel_x + 10, panel_y + 40), 
                           self.font, self.font_scale, self.text_color, self.font_thickness)
                
                cv2.putText(self.display_image, f"BGR: ({b}, {g}, {r})", 
                           (panel_x + 10, panel_y + 60), 
                           self.font, self.font_scale, self.text_color, self.font_thickness)
                                
                # Update display
                cv2.imshow('Image', self.display_image)
                
            except Exception as e:
                print(f"Error processing pixel: {e}")
                # If there's an error, fallback to a simple display
                self.display_image = self.image.copy()
                cv2.putText(self.display_image, f"Coords: ({x}, {y})", 
                           (x + 10, y + 20), 
                           self.font, self.font_scale, (0, 0, 255), self.font_thickness)
                cv2.imshow('Image', self.display_image)
    
    def run(self):
        print("Click on the image to see pixel coordinates and BGR values.")
        print("Press 'q' to quit.")
        
        # Main loop
        while True:
            cv2.imshow('Image', self.display_image)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
        
        cv2.destroyAllWindows()
        # Force window closing in Jupyter
        for i in range(5):
            cv2.waitKey(1)

In [3]:
# image_path = "images/nabi.jpeg" # Sample image
image_path = input("Enter the path to the image: ")

image_viewer = ImageClickDisplay(image_path)
image_viewer.run()

Click on the image to see pixel coordinates and BGR values.
Press 'q' to quit.


2025-02-26 00:34:43.953 Python[34905:21066261] +[IMKClient subclass]: chose IMKClient_Legacy
2025-02-26 00:34:43.953 Python[34905:21066261] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
